In [1]:
import sys
!{sys.executable} -m pip install lyricsgenius
!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install selenium
import pandas as pd
import requests
import lyricsgenius as genius
import spotipy
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
genius_client_id = 'Qeii-UKMUUwsAYCuuemsF7CZk7R2bFDhjP1FMkqpozASvYsA4wQDlLmd2JFM3Vhm'
genius_secret_id = 'fWhV7VtwAL-5D32Ga9Y0Kyh_VFEo7XK0Jf8-3RMcZ5EoMXqsDpmFWFFPyobnibfMXJuJ-GZe69YEuXR-36VGAA'
genius_client_access_token = '1QVL91jZbgdOIhvYhqKku_zkFDf6MHEyLpXEITHU2tE2HGvAqYIPInTrQaWcInD7'
app_website_url = 'http://cis545genius.com'
redirect_uri = 'cis545genius.com'
base_url = 'https://api.genius.com'
driver_path = '/Users/ericdong/Downloads/chromedriver'
# driver_path = "/mnt/c/Users/jeffr/Downloads/chromedriver_win32/chromedriver.exe"
artist_url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
page_downs = 0
api = genius.Genius(genius_client_access_token)

spotify_client_id = '96bdc763e3b5446ebbd6c44850055bcb'
spotify_secret_id = 'f7a5054823ed491aa1856ecdc7849062'


In [3]:
def get_artists(url, page_downs, driver_path):
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    time.sleep(1)
    page = driver.find_element_by_tag_name('body')
    for i in range(page_downs):
        page.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
    artists = driver.find_elements_by_class_name('listItem__data')
    names = [artist.find_element_by_class_name('listItem__title').text for artist in artists[:-1]]
    driver.quit()
    
    return names

In [12]:
def get_songs(artist):
    return api.search_artist(artist, verbose=False, max_songs=10)

In [5]:
artists = get_artists(artist_url, page_downs, driver_path)

In [7]:
print(artists)

['Tupac', 'Eminem', 'The Notorious B.I.G.', 'Nas', 'Ice Cube', 'Jay-Z', 'Snoop Dogg', 'Dr. Dre', 'Kendrick Lamar', 'Rakim', 'André 3000', 'Eazy-E', '50 Cent', 'Kanye West', 'DMX', 'J. Cole', 'Busta Rhymes', 'Method Man', 'Mos Def', 'Lil Wayne', 'Ludacris', 'KRS-One', 'LL Cool J', 'Common', 'Ghostface Killah', 'Big L', 'Redman', 'T.I.', 'Big Pun', 'Nate Dogg', 'Tech N9ne', 'Lauryn Hill', 'Scarface', 'Slick Rick', 'The Game', 'Raekwon', 'Big Daddy Kane', "Ol' Dirty Bastard", 'Mobb Deep', 'Chance the Rapper', 'Logic', 'Cypress Hill', 'Ice-T', 'Lupe Fiasco', 'RZA', 'GZA', 'Warren G', 'Q-Tip', 'Xzibit', 'ASAP Rocky']


In [8]:
# test set of artists
artists = ['Kanye West', 'Migos']

In [13]:
music = []
for artist in artists:
    try:
        music = music + get_songs(artist).songs
    except AssertionError:
        pass

In [14]:
print(music)

[("'03 Electric Relaxation", 'Kanye West'), ('18 Years', 'Kanye West'), ('1996 Fat Beats Freestyle', 'Kanye West'), ('2002 Unreleased Freestyle', 'Kanye West'), ('2004 Tim Westwood Freestyle', 'Kanye West'), ('2013 Yeezus Tour Dates', 'Kanye West'), ('2013 Zane Lowe BBC Radio Interview Part 1', 'Kanye West'), ('2013 Zane Lowe BBC Radio Interview Part 2', 'Kanye West'), ('2014 Yeezus Tour Dates', 'Kanye West'), ('2015 BET Honors Visionary Award Acceptance Speech', 'Kanye West'), ('1017', 'Migos'), ('11 Birds', 'Migos'), ('2Pac & Biggie', 'Migos'), ('3 Mics', 'Migos'), ('3 Way EP [Album Art + Tracklist]', 'Migos'), ('3 Way (Intro)', 'Migos'), ('50 Bitches (Lick It Off)', 'Migos'), ('50 Chicks', 'Migos'), ('9 On Me', 'Migos'), ('Add It Up', 'Migos')]


In [17]:
df = pd.DataFrame()
df['artist'] = list(map(lambda x: x.artist, music))
df['song'] = list(map(lambda x: x.title, music))
df['album'] = list(map(lambda x: x.album, music))
df['lyrics'] = list(map(lambda x: x.lyrics, music))
df.head()

,artist,song,album,lyrics
0,Kanye West,'03 Electric Relaxation,None,You got to\n\nRelax yourself or we'll whack ya...
1,Kanye West,18 Years,Def Poetry Jam,"The next guest.. is my man\nBringing him back,..."
2,Kanye West,1996 Fat Beats Freestyle,None,"Yo it's the Lake Shore Drive\nKeep it live, ti..."
3,Kanye West,2002 Unreleased Freestyle,None,"Ayo\nPlanes, trains, automobiles\nThe G500s wi..."
4,Kanye West,2004 Tim Westwood Freestyle,None,I told her to drive over after dude crib\nBrin...


In [33]:
# list(map(lambda x: x.annotation, music))

In [18]:
# headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
# search_url = base_url + '/search'
# song_title = 'Mercy'
# params = {'q': song_title}
# response = requests.get(search_url, params=params, headers=headers)
# json = response.json()
# print(json)

In [21]:
df.to_pickle('data.pkl')

In [34]:
spotify_token = 'BQCVEzRlbQgkEb06zkDYJKDQAzzGCm19cyf5pugp5RnUxls0OyJR-vlNyxa8bjIx-_vaBkXrY7FrSxMQCGsGieSXVgE8VnVlCtiNm-nNSnRDNsayJWUaMgKQwAk4iNisv_i6vx4FzpE'
spotify = spotipy.Spotify(auth=spotify_token)
spotify_artists = [spotify.search(q='artist:' + artist, type='artist') for artist in artists]

In [35]:
print(spotify_artists)

[{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AKanye+West&type=artist&market=US&offset=0&limit=10', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'}, 'followers': {'href': None, 'total': 7141349}, 'genres': ['pop', 'pop rap', 'rap'], 'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x', 'id': '5K4W6rqBFWDnAN6FQUkS6x', 'images': [{'height': 1000, 'url': 'https://i.scdn.co/image/651370b09316c3851153b1ccf04e7d6f29bf39df', 'width': 1000}, {'height': 640, 'url': 'https://i.scdn.co/image/69813307d0aabd163bd206455fd1d8a45484e2d1', 'width': 640}, {'height': 200, 'url': 'https://i.scdn.co/image/f30e0dfe57e25c2ee133b15242a7fc0eb917232c', 'width': 200}, {'height': 64, 'url': 'https://i.scdn.co/image/78ff00bb0c95d64f016739628bac1198b0ac6e98', 'width': 64}], 'name': 'Kanye West', 'popularity': 89, 'type': 'artist', 'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}, {'external_urls': {'spotify': 'https://open.spot

In [36]:
yeezy = spotify.artist_top_tracks('5K4W6rqBFWDnAN6FQUkS6x')
for track in yeezy['tracks'][:10]:
    print('track    : ' + track['name'])

track    : Stronger
track    : FourFiveSeconds
track    : Gold Digger
track    : POWER
track    : Ni**as In Paris
track    : Father Stretch My Hands Pt. 1
track    : Waves
track    : Famous
track    : Heartless
track    : All Of The Lights
